## Import modules

In [1]:
import numpy as np
import os
import PIL
from PIL import Image

from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

C:\Users\HP\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Reading data

In [3]:
import numpy as np
train = np.load('C:\\Piazza\\Activity Lab\\CNN-Lab\\cifar10_cnn1\\cifar10_cnn\\cifar_train.npz')
x_train = train['data']
train_labels = train['labels']

test = np.load('C:\\Piazza\\Activity Lab\\CNN-Lab\\cifar10_cnn1\\cifar10_cnn\\cifar_test.npz')
x_test = test['data']
test_labels = test['labels']

print('train_data:', x_train.shape)
print('train_labels:', train_labels.shape)
print('test_data:', x_test.shape)
print('test_labels:', test_labels.shape)

train_data: (50000, 32, 32, 3)
train_labels: (50000, 1)
test_data: (10000, 32, 32, 3)
test_labels: (10000, 1)


In [3]:
batch_size = 64
nb_classes = 10

img_rows, img_cols = 32, 32    # input image dimensions
img_channels = 3               # The CIFAR10 images are RGB.

# Convert class vectors to binary class matrices.
from keras.utils import to_categorical
y_train = to_categorical(train_labels, nb_classes)
y_test = to_categorical(test_labels, nb_classes)

print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


## Resize all the images 

In [4]:
def resize(p, size):
    return Image.fromarray(p).resize(size=(size,size))

def resize_all(arr, size):
    t = []
    for i in range(arr.shape[0]):
        t.append(np.array(resize(arr[i], size)))

    return(np.array(t))

In [5]:
x_train = resize_all(x_train, 224)
x_test = resize_all(x_test, 224)

In [6]:
# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

In [7]:
print(x_train.shape)
print(x_test.shape)

(50000, 224, 224, 3)
(10000, 224, 224, 3)


## CNN Architecture

In [8]:
# Get Inception architecture from keras.applications
from keras.applications.inception_v3 import InceptionV3, GlobalAveragePooling2D

trained_model = InceptionV3(include_top=False,weights='imagenet')

x = trained_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
pred_inception= Dense(10,activation='softmax')(x)
model = Model(inputs=trained_model.input,outputs=pred_inception)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
#making the layers of vgg16 non-trainable
for layer in trained_model.layers:
    layer.trainable=False
    
#compiling the model
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=adam)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Training 

In [10]:
nb_epoch = 2
model.fit(x_train[:5000], y_train[:5000],
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_data=(x_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 228s - loss: 0.8907 - acc: 0.7034 - val_loss: 0.7739 - val_acc: 0.7437
Epoch 2/2
50000/50000 [==============================] - 187s - loss: 0.7291 - acc: 0.7529 - val_loss: 0.7491 - val_acc: 0.7527


In [ ]:
model.save('cifar10_transferlearning.h5')
# model.save_weights('cifar10_transferlearning.h5')